## PART 1

In [ ]:
from analysis.sim_import import parse_trips_file, parse_legs_file
from analysis.utils import load_df_to_database, load_db_parameters

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
trips = 'Y:/stuttgart-v1.0/05_stuttgart-v1.0_scenariotest/output/output-m5_2/m5_2.output_trips.csv.gz'
db_parameter = 'C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/03_analysis/01_calibration/db_param_10pct.json'
db_parameter = load_db_parameters(db_parameter)

In [ ]:
gdf = parse_trips_file(trips)

In [ ]:
gdf.head(3)

In [ ]:
gdf.loc[1144]

In [ ]:
sample = gdf.sample(n=3)
sample

In [ ]:
table_name = 'sim_trips_raw'
table_schema = 'matsim_output'

DATA_METADATA = {
    'title': 'Trips',
    'description': 'Trip table',
    'source_name': 'Senozon Input',
    'source_url': 'Nan',
    'source_year': '2020',
    'source_download_date': 'Nan',
}

In [ ]:
load_df_to_database(
    df=sample,
    update_mode='append',
    db_parameter=db_parameter,
    schema=table_schema,
    table_name=table_name,
    meta_data=DATA_METADATA,
    geom_cols={'geometry': 'LINESTRING'})

## PART 2

In [ ]:
import pandas as pd

In [ ]:
calib = 'C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/01_prep/05_CalibrationData/Calibration_Data_v2.xlsx'
table = pd.read_excel(calib, sheet_name='05_OEVSegmente', skipfooter=10)

In [ ]:
table

## PART 3

In [1]:
import pandas as pd
import geopandas as gpd
from matsim import read_network

In [2]:
network = 'C:/Users/david/Desktop/bc.output_network.xml.gz'

In [3]:
network = read_network(network, True)
gdf_network = network.as_geo()
gdf_network = gdf_network.set_crs(epsg=25832)

In [8]:
network.link_attrs

""


In [ ]:
gdf_network.head(3)

## PART 4

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
network = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/02_runs/stuttgart-v1.0/05_stuttgart-v1.0_scenariotest/output/output-m5_1/network-shp/m5_1.network.shp"
gdf = gpd.read_file(network)

In [4]:
gdf.head(3)

,Id,Length,capacity,lanes,Freespeed,Modes,zone_name,zone_group,h_costs,dmax_costs,max_time,penalty,res_costs,geometry
0,10000014,50.53,1200.0,2.0,9.47,"car,ride,walk,bike",None,None,0.0,0.0,30.0,0.0,0.0,"LINESTRING (477508.123 5466441.289, 477459.281..."
1,10000015,61.02,1200.0,2.0,9.56,"car,ride,walk,bike",None,None,0.0,0.0,30.0,0.0,0.0,"LINESTRING (477508.123 5466441.289, 477557.847..."
2,10000026,61.02,1200.0,2.0,9.52,"car,ride,walk,bike",None,None,0.0,0.0,30.0,0.0,0.0,"LINESTRING (477557.847 5466406.321, 477508.123..."


In [5]:
gdf.crs

<Projected CRS: EPSG:25832>
Name: ETRS89 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe - 6°E to 12°E and ETRS89 by country
- bounds: (6.0, 38.76, 12.0, 83.92)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## PART 5

In [1]:
import xml.etree.ElementTree as ET
import gzip
import pandas as pd
import shapely as shp
import geopandas as gpd

In [2]:
tSFile = "C:/Users/david/Desktop/m5_ex.output_transitSchedule.xml.gz"

In [12]:
transitStops = list()

try:
    with gzip.open(tSFile) as f:
        tree = ET.parse(f)
        root = tree.getroot()
        
        for tSF in root.find('./transitStops'):
               
        transitStops.append({
            "id": tSF.attrib['id'],
            "x": tSF.attrib['x'],
            "y": tSF.attrib['y'],
            "link_ref_id": tSF.attrib['linkRefId'],
            "name": tSF.attrib['name'],
            "is_blocking": tSF.attrib['isBlocking'],
            "vvs_bike_ride": tSF.find('./attributes/attribute[@name="VVSBikeAndRide"]').text
                  
            })
                
            
except OSError as e:
    raise Exception(e.strerror)
    
transitStops = pd.DataFrame(transitStops)

false


In [5]:
transitStops = gpd.GeoDataFrame(transitStops, geometry=gpd.points_from_xy(transitStops.x, transitStops.y))

,id,x,y,linkRefId,name,isBlocking,geometry
0,100185,503026.79157291225,5481587.615674013,tr372608,"Gaimühle Sägewerk, Eberbach",false,POINT (503026.792 5481587.616)
1,100185.1,503026.79157291225,5481587.615674013,tr372621,"Gaimühle Sägewerk, Eberbach",false,POINT (503026.792 5481587.616)
2,100186,498323.42781285214,5489709.952489486,tr356468,"Beerfelden Seeweg, Oberzent",false,POINT (498323.428 5489709.952)
3,100186.1,498323.42781285214,5489709.952489486,tr356481,"Beerfelden Seeweg, Oberzent",false,POINT (498323.428 5489709.952)
4,100186.2,498323.42781285214,5489709.952489486,tr356490,"Beerfelden Seeweg, Oberzent",false,POINT (498323.428 5489709.952)
...,...,...,...,...,...,...,...
52411,941089.1,466277.8405306667,5410371.3784105275,tr479306,"Schwann Kalkstatt, Straubenhardt",false,POINT (466277.841 5410371.378)
52412,941091,477044.6313894393,5406327.62251384,tr507136,"Kapfenhardt Hausackerweg, Unterreichenbach",false,POINT (477044.631 5406327.623)
52413,941092,476863.51338851167,5406100.484584558,tr507137,"Kapfenhardt Linde, Unterreichenbach",false,POINT (476863.513 5406100.485)
52414,941092.1,476863.51338851167,5406100.484584558,tr543758,"Kapfenhardt Linde, Unterreichenbach",false,POINT (476863.513 5406100.485)


## PART 6

In [1]:
import pandas as pd
import geopandas as gpd
import matsim

In [2]:
plans_str = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/03_MATSim/02_runs/stuttgart-v1.0/01_stuttgart-v1.0_fstRun01/input/stuttgart-v1.0-10pct.plans.xml.gz"

In [5]:
plans = matsim.plan_reader(plans_str, selectedPlansOnly=True)

for person, plan in plans:
    no = 0
    print(person.attrib['id'])
    for e in plans:
        no = no + 1       
        

67166005_456


In [8]:
pd.DataFrame(activities)

""


In [ ]:
    no = 0
    for a in list(acts):
            
        if no != 0:
            no = no + 1
            activities.append({'person_id': p_id,
                               'act_no': act_no,
                               'act_type': act_type,
                               'trigger': "end",
                               'x': x,
                               'y': y})

        p_id = person.attrib['id']
        act_no = no
        act_type = a.attrib['type']
        x = a.attrib['x']
        y = a.attrib['y']

        if no != 0:
            activities.append({'person_id': p_id,
                               'act_no': act_no,
                               'act_type': act_type,
                               'trigger': "start",
                               'x': x,
                               'y': y})



df_activities = 
#gdf_activities = gpd.GeoDataFrame(df_activities.drop(columns=['x', 'y']),
                                #geometry=gpd.points_from_xy(df_activities.x, df_activities.y))
#gdf_activities = gdf_activities.set_crs(epsg=25832)